This notebook will be used for the IBM Data Science Certificate capstone project, "Battle of the Neighbourhoods".

In [52]:
import pandas as pd
import numpy as np

In [53]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Week 3 Assignment

In [54]:
from bs4 import BeautifulSoup
import requests

In [113]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html = requests.get(wiki_url).content
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table')

In [118]:
df_toronto = pd.read_html(str(table))[0]

**The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood**

In [119]:
df_toronto.columns = ['PostalCode','Borough','Neighborhood']

**Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.**

In [124]:
df_toronto = df_toronto[df_toronto['Borough'] != 'Not assigned']

**More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.**

In [130]:
# Proof that there are no duplicates in the PostalCode column
if len(list(df_toronto['PostalCode'])) == len(set(df_toronto['PostalCode'])):
    print('No duplicates in PostalCode column')
else:
    print('Possible duplicates in PostalCode column')

No duplicates in PostalCode column


**If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.**

In [129]:
df_toronto[df_toronto['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


**Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.**

**In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.**

In [127]:
df_toronto.shape

(103, 3)